### PIPELINE

In [10]:
import cv2
import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm



# Add the 'src' directory to the Python path
sys.path.append(os.path.join(os.getcwd(), 'src'))
import Evaluator
import Utils



In [2]:


# Define image  names
images = ["0_empty", "0","1","2","3","4","5","6","7","8","9"]
#images =["1"]

# dictonary for two different sets of parameters
parameters = {
    "over_130": {
        "noise_reduction":  "Gaussian",  
        "filter_size":  3, 
        "clahe":  True, 
        "grid_size":  8,  
        "pct":  50.0,
        "static_foreground_removal":  True,
        "dilation":  True,
        "opening":  False,
        "closing":  True,
        "smoothing_after_morph":  True, 
         
    },
    "below_130": {
        "noise_reduction":  "Median",  
        "filter_size":  3, 
        "clahe":  False, 
        "grid_size":  8,  
        "pct":  35.0,
        "static_foreground_removal":  True,
        "dilation":  True,
        "opening":  True,
        "closing":  True,
        "smoothing_after_morph":  False, 
    }
}







In [3]:

# Initialize the DataFrame with specific column types
df_eval = pd.DataFrame({
    'image': pd.Series(dtype='str'),
    'objects_before_post_processing': pd.Series(dtype='int'),
    'objects_after_post_processing': pd.Series(dtype='int'),
    'TP': pd.Series(dtype='int'),
    'FP': pd.Series(dtype='int'),
    'FN': pd.Series(dtype='int'),
    'Precision': pd.Series(dtype='float'),
    'Recall': pd.Series(dtype='float'),
    'F1': pd.Series(dtype='float'),
    'RMSE': pd.Series(dtype='float')
})

In [4]:
# Function to process each parameter combination
def process_image(image):
    
    
    # Load the image
    img = cv2.imread(f"data/images/{image}.jpg")
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load labels
    labels = pd.read_csv(f"data/labels/labels_{image}.csv")
    labels.columns = ['Label', 'X', 'Y', 'Image', 'Width', 'Height']
    actual_persons = len(labels)
    labels_np = labels[['X', 'Y']].to_numpy()

    # Calulate avg intensity value
    avg_intensity = np.mean(img_gray)

    # if the avg intensity is above 130, use the set of parameters for images with high intensity

    # Select parameters based on intensity
    if avg_intensity > 130:
        params_set = "over_130"
    else:
        params_set = "below_130"

    # Extract parameters
    noise = parameters[params_set]["noise_reduction"]
    size = parameters[params_set]["filter_size"]
    clahe = parameters[params_set]["clahe"]
    grid = parameters[params_set]["grid_size"]
    p = parameters[params_set]["pct"]
    static_removal = parameters[params_set]["static_foreground_removal"]
    d = parameters[params_set]["dilation"]
    o = parameters[params_set]["opening"]
    c = parameters[params_set]["closing"]
    smooth = parameters[params_set]["smoothing_after_morph"]


    # Preprocess the image
    image_preprocessed = Utils.preprocess_image(img, noise, size, clahe, grid)

    # Background subtraction
    image_no_background = Utils.background_subtraction(image_preprocessed, p, static_removal)

    # Morphological operations
    image_morph = Utils.apply_morphological_operations(image_no_background, d, o, c, smooth)

    # Segment objects with connected components
    num_labels, labels_im, stats, centroids = cv2.connectedComponentsWithStats(image_morph, connectivity=8)
    num_objects_before_post_processing = num_labels - 1

    # Post processing
    bounding_boxes = Utils.post_process_image(image_morph, stats, centroids, nms_threshold=0.5)

    # Evaluate the results
    eval = Evaluator.Evaluator()
    eval.evaluate_bounding_boxes(bounding_boxes, labels_np)

    # get bounding boxes
    tp_boxes, tp_points, fp_boxes, fn_points = eval.get_bounding_boxes()

    # draw result
    #Utils.draw_result(img, tp_boxes, tp_points, fp_boxes, fn_points)
    output_path = f"results/images/{image}_result.jpg"
    img= Utils.draw_result(img, tp_boxes, tp_points, fp_boxes, fn_points)
    # save the result image as rgb
    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # Prepare row data
    row = {
        'Image': image, 
        'TP': eval.TP, 
        'FP': eval.FP, 
        'FN': eval.FN, 
        'Precision': eval.get_precision(), 
        'Recall': eval.get_recall(), 
        'F1': eval.get_f1_score(), 
        'RMSE': eval.get_RMSE(actual_persons, eval.TP + eval.FP)
    }
    
    return row

In [5]:

all_results = []
# Initialize the progress bar with the total number of parameter combinations
with tqdm(total=len(images), desc="Progress") as pbar:
    # Iterate over each image and process combinations sequentially
    for image in images:

        result = process_image(image)
                
        if result:  
            all_results.append(result)
            
            
        pbar.update(1)  
all_results = pd.DataFrame(all_results)
   

Progress:   0%|          | 0/11 [00:00<?, ?it/s]

Progress: 100%|██████████| 11/11 [01:00<00:00,  5.51s/it]


In [6]:
all_results

,Image,TP,FP,FN,Precision,Recall,F1,RMSE
0,0_empty,0,4,0,0.000000,1.000000,0.000000,4.0
1,0,1,3,0,0.250000,1.000000,0.400000,3.0
2,1,11,12,11,0.478261,0.500000,0.488889,1.0
3,2,28,13,23,0.682927,0.549020,0.608696,10.0
4,3,99,62,35,0.614907,0.738806,0.671186,27.0
5,4,105,91,30,0.535714,0.777778,0.634441,61.0
6,5,127,74,44,0.631841,0.742690,0.682796,30.0
7,6,142,64,39,0.689320,0.784530,0.733850,25.0
8,7,152,31,81,0.830601,0.652361,0.730769,50.0
9,8,182,27,63,0.870813,0.742857,0.801762,36.0
